In [1]:
import requests

In [2]:
url='**'
response=requests.get(url)

MissingSchema: Invalid URL '**': No schema supplied. Perhaps you meant http://**?

In [ ]:
pprint(response.content)

(b'{"response": {}, "meta": {"url": "https://api.ycharts.com/v3/companies/AAPL,'
 b'DDR/splits?start_date=2004-10-10&end_date=2015-01-12", "status": "error", "e'
 b'rror_code": 401, "error_message": "Missing/Invalid Authorization token."}}')


In [4]:

import yfinance as yf
import mplfinance as mpf 
import numpy as np

ticker = "AAPL"
data = yf.download(ticker, start="2021-08-01", end="2022-09-1")
mpf.plot(data)
mpf.plot(data, type='candle')

mpf.plot(data, type='candle', mav=8)


ModuleNotFoundError: No module named 'yfinance'

In [ ]:
mpf.plot(data, type='candle', mav=(8, 20, 50))


In [ ]:
mpf.plot(data, type='candle', mav=(8, 20, 50), volume=True)



In [ ]:

data['MA20'] = data['Adj Close'].rolling(20).mean()
data['MA50'] = data['Adj Close'].rolling(50).mean()
data = data.dropna()

buy_signals = []
sell_signals = []

for i in range(len(data)):
    if (data['MA20'].iloc[i] > data['MA50'].iloc[i]) and (data['MA20'].iloc[i-1] < data['MA50'].iloc[i-1]):
        buy_signals.append(data.iloc[i]['Adj Close'] * 0.98)
    else:
        buy_signals.append(np.nan)
    
    if (data['MA20'].iloc[i] < data['MA50'].iloc[i]) and (data['MA20'].iloc[i-1] > data['MA50'].iloc[i-1]):
        sell_signals.append(data.iloc[i]['Adj Close'] * 1.02)
    else:
        sell_signals.append(np.nan)

buy_markers = mpf.make_addplot(buy_signals, type='scatter', markersize=120, marker='^')
sell_markers = mpf.make_addplot(sell_signals, type='scatter', markersize=120, marker='v')
apds = [buy_markers, sell_markers]
mpf.plot(data, type="candle", addplot=apds)

In [ ]:

import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import getFamaFrenchFactors as gff

ticker = "AMZN"
stock_data = yf.download(ticker, start="2016-06-30", end="2021-07-31")

stock_prices = stock_data['Adj Close']
stock_prices = stock_prices.resample("1M").last()
returns = stock_prices.pct_change()
returns = returns.dropna()
returns.name = "Asset"

ff3_monthly = pd.DataFrame(gff.famaFrench3Factor(frequency='m'))
ff3_monthly.rename(columns={"date_ff_factors": 'Date'}, inplace=True)
ff3_monthly.set_index('Date', inplace=True)
ff_data = ff3_monthly.merge(returns, on='Date')
rf = ff_data['RF'].mean()
market_premium = ff_data['Mkt-RF'].mean()

X = ff_data['Mkt-RF']
ff_data['Asset-RF'] = ff_data['Asset'] - ff_data['RF']
y = ff_data['Asset-RF']
c = sm.add_constant(X)
capm_model = sm.OLS(y, c)
result = capm_model.fit()
print(result.summary())
intercept, beta = result.params

expected_return = rf + beta*market_premium
print("Expected monthly returns: " + str(expected_return))
yearly_return = expected_return * 12
print("Expected yearly returns: " + str(yearly_return))

In [ ]:
from pip import config

In [3]:
from decouple import config

In [4]:
sec = config('sec')
sec

'd297523f230577b078270fbe8c42f1ae12973361e66624885c5a0aa4e09e01d8'

In [7]:
from sec_api import QueryApi

queryApi = QueryApi(api_key=sec)

query = {
  "query": { "query_string": { 
      "query": "ticker:TSLA AND filedAt:{2020-01-01 TO 2022-12-31} AND formType:\"10-Q\"" 
    } },
  "from": "0",
  "size": "10",
  "sort": [{ "filedAt": { "order": "desc" } }]
}

filings = queryApi.get_filings(query)

pprint(filings)

{'filings': [{'accessionNo': '0000950170-21-002253',
              'cik': '1318605',
              'companyName': 'Tesla, Inc.',
              'companyNameLong': 'Tesla, Inc. (Filer)',
              'dataFiles': [{'description': 'XBRL TAXONOMY EXTENSION '
                                            'CALCULATION LINKBASE DOCUMENT',
                             'documentUrl': 'https://www.sec.gov/Archives/edgar/data/1318605/000095017021002253/tsla-20210930_cal.xml',
                             'sequence': '6',
                             'size': '75631',
                             'type': 'EX-101.CAL'},
                            {'description': 'XBRL TAXONOMY EXTENSION '
                                            'PRESENTATION LINKBASE DOCUMENT',
                             'documentUrl': 'https://www.sec.gov/Archives/edgar/data/1318605/000095017021002253/tsla-20210930_pre.xml',
                             'sequence': '7',
                             'size': '538277',
         

In [6]:
from pprint import pprint

In [ ]:
import pandas as pd
  
# Nested dictionary to convert it
# into multiindex dataframe
nested_dict = {'A': {'a': [1, 2, 3,
                           4, 5],
                     'b': [6, 7, 8,
                           9, 10]},
  
               'B': {'a': [11, 12, 13,
                           14, 15],
                     'b': [16, 17, 18,
                           19, 20]}}
  
reformed_dict = {}
for outerKey, innerDict in nested_dict.items():
    for innerKey, values in innerDict.items():
        reformed_dict[(outerKey,
                       innerKey)] = values
  
# Multiindex dataframe

multiIndex_df = pd.DataFrame(reformed_dict)
multiIndex_df

A       B    
   a   b   a   b
0  1   6  11  16
1  2   7  12  17
2  3   8  13  18
3  4   9  14  19
4  5  10  15  20